**Part a)** The first part is to split the data into three sets: training, validation, and test. Split each class separately so that there is an equal percentage of each class in all three sets. The number of total images in the validation is to be roughly 2000, in the test set 3000, and the remainder for the training set. It does not need to be exact. This is called a stratified split.

In [43]:
from pathlib import Path
import shutil
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

import torch.optim as optim
from ResNet import ResNet

from sklearn.metrics import accuracy_score, average_precision_score

import matplotlib.pyplot as plt
import pandas as pd

device = torch.device('cuda') if torch.cuda.is_available else 'cpu'

First, we decide the classes. `.parts[-1]` gets out the last part in the directory, whilst `.iterdir()` iterates through directories, and `.is_dir()` sees to it, that it really is a directory.

In [44]:
dataset = Path('/mnt/e/ml_projects/IN3310/2025/tut_data/mandatory1_data/')

classes = [str(subdir.parts[-1]) for subdir in dataset.iterdir() if subdir.is_dir()]
classes

['forest', 'buildings', 'sea', 'glacier', 'mountain', 'street']

Then, we create directories for `train, val, test` in the root folder

In [45]:
base_path = Path('/mnt/e/ml_projects/IN3310/2025/tut_data/oblig1/')

# the directories we want to create
dirs = ['train', 'val', 'test']

for dir_name in dirs:
    # creating a path string
    dir_path = base_path / dir_name
    dir_path.mkdir(parents=True, exist_ok=True) # creating directory

    # creating subdirectories of class names
    for class_name in classes:
        class_path = base_path / dir_name / class_name
        class_path.mkdir(parents=True, exist_ok=True)

Now we need to split the data into train, test, vals. A good way of doing this is to use `train_test_split` of the filenames and classes, and then fill our folders up.

In [46]:
img_paths = [] # container for image paths
class_indices = [] # container for class indices

for class_index, class_name in enumerate(classes):
    class_path = dataset / class_name
    for img_file in class_path.iterdir():
        if img_file.is_file():
            img_paths.append(img_file)
            class_indices.append(class_index)

In [47]:
train_imgs, temp_imgs, train_indices, temp_indices = train_test_split(
    img_paths, class_indices, test_size=0.3, stratify=class_indices, random_state=42
)

In [48]:
val_imgs, test_imgs, val_indices, test_indices = train_test_split(
    temp_imgs, temp_indices, test_size=0.6, stratify=temp_indices, random_state=42
)

In [49]:
def copy_images(img_paths, class_indices, split_name):
    
    for img_path, class_index in zip(img_paths, class_indices):
        target_dir = base_path / split_name / classes[class_index]
        target_file = target_dir / img_path.name

        # copying the file if it's not already there
        if not target_file.exists():
            shutil.copy(img_path, target_file)

In [50]:
copy_images(train_imgs, train_indices, 'train')
copy_images(val_imgs, val_indices, 'val')
copy_images(test_imgs, test_indices, 'test')

We now have folders with the data. Finally we can do a check for duplicates.

**Part b)** Create a solution to verify that the dataset splits are disjoint. Ensure that no file appears in more than one of your training, validation, or
test sets.

In [51]:
def verify_no_duplicates(train_imgs, val_imgs, test_imgs):
    train_set = set(train_imgs)
    val_set = set(val_imgs)
    test_set = set(test_imgs)

    # using intersection to check for data overlaps
    assert len(train_set.intersection(val_set)) == 0, 'Overlap between Train and Val'
    assert len(train_set.intersection(test_set)) == 0, 'Overlap between Train and Test'
    assert len(val_set.intersection(test_set)) == 0, 'Overlap between Val and Test'

verify_no_duplicates(train_imgs, val_imgs, test_imgs)

**Part c)** Develop and implement dataloaders for training, validation, and test sets. Please make one root path for the dataset, this makes it easier for us
to check/debug your work. If there are multiple paths to the dataset that we need to change, it becomes tricky to change them all.

In [53]:
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(), # converts to a torch tensor
    transforms.Normalize((0.5,), (0.5,)) # normalizes to [-1, 1]
])

augment_transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomHorizontalFlip(p=0.5),  # flipping image (P=50%)
    transforms.RandomRotation(15),  # rotating ±15 degrees
    transforms.RandomResizedCrop(150, scale=(0.8, 1.0)),  # cropping randomly and scaling
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Endrer farger
    transforms.ToTensor(),  
    transforms.Normalize((0.5,), (0.5,))  
])

In [54]:
train_dataset = datasets.ImageFolder(root=base_path / 'train', transform=transform)
train_dataset_augm = datasets.ImageFolder(root=base_path / 'train', transform=augment_transform)
val_dataset = datasets.ImageFolder(root=base_path / 'val', transform=transform)
test_dataset = datasets.ImageFolder(root=base_path / 'test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
train_loader_augm = DataLoader(train_dataset_augm, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

2b) Use the dataloaders you created in Part 1 to feed
the training data into the model of your choosing. Write code to perform the training process, ensuring that the model is optimized over the training data. Make sure to use the validation dataset to monitor performance during training. During training, monitor the model’s performance using accuracy on the validation set. This will give you an initial indication of how well your model is learning.

In [31]:
def evaluate_model(model, dataloader):
    model.eval() # put model in evaluation mode

    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            probs = torch.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)

            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())
            all_probs.append(probs.cpu())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    all_probs = torch.cat(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)

    ap_scores = []
    for i in range(all_probs.shape[1]):
        binary_labels = (all_labels == i).float()

        ap = average_precision_score(
            binary_labels.cpu().numpy(),
            all_probs[:, i].cpu().numpy()
        )
        ap_scores.append(ap)

    map_score = sum(ap_scores) / len(ap_scores)
    
    return accuracy, ap_scores, map_score


In [35]:
def train_model(model, train_loader, val_loader, criterion, optimizer, lr, num_epochs=10):
    model = model.to(device)
    optimizer = optimizer(model.parameters(), lr=lr)

    train_accs = []
    val_accs = []
    map_scores = []

    for epoch in range(num_epochs):
        model.train() # putting model into training mode
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad() # zeroing out the optimizer
            
            outputs = model(images)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_accuracy = correct / total
        val_accuracy, _, map_score = evaluate_model(model, val_loader)

        train_accs.append(train_accuracy)
        val_accs.append(val_accuracy)
        map_scores.append(map_score)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Accuracy: {train_accuracy:.4f}, Val Accuracy: {val_accuracy:.4f}, mAP: {map_score:.4f}")

    return train_accs, val_accs, map_scores

        
    

In [ ]:
criterion = nn.CrossEntropyLoss()
model = ResNet(img_channels=3, num_layers=34, num_classes=len(classes))
train_acc1, val_acc1, map_scores1 = train_model(model, train_loader, val_loader, criterion, optim.Adam, 0.001, 15)


Epoch 1/20, Train Accuracy: 0.6216, Val Accuracy: 0.5093
Epoch 2/20, Train Accuracy: 0.7569, Val Accuracy: 0.7882
Epoch 3/20, Train Accuracy: 0.7922, Val Accuracy: 0.7055
Epoch 4/20, Train Accuracy: 0.8094, Val Accuracy: 0.8322
Epoch 5/20, Train Accuracy: 0.8365, Val Accuracy: 0.8209
Epoch 6/20, Train Accuracy: 0.8435, Val Accuracy: 0.7104
Epoch 7/20, Train Accuracy: 0.8551, Val Accuracy: 0.8112
Epoch 8/20, Train Accuracy: 0.8640, Val Accuracy: 0.8420
Epoch 9/20, Train Accuracy: 0.8709, Val Accuracy: 0.8190
Epoch 10/20, Train Accuracy: 0.8800, Val Accuracy: 0.8415
Epoch 11/20, Train Accuracy: 0.8912, Val Accuracy: 0.8390
Epoch 12/20, Train Accuracy: 0.8989, Val Accuracy: 0.8513
Epoch 13/20, Train Accuracy: 0.9114, Val Accuracy: 0.8591
Epoch 14/20, Train Accuracy: 0.9207, Val Accuracy: 0.8249
Epoch 15/20, Train Accuracy: 0.9288, Val Accuracy: 0.8601
Epoch 16/20, Train Accuracy: 0.9389, Val Accuracy: 0.8567
Epoch 17/20, Train Accuracy: 0.9481, Val Accuracy: 0.8547
Epoch 18/20, Train Accu

In [ ]:
model = ResNet(img_channels=3, num_layers=34, num_classes=len(classes))
train_acc2, val_acc2, map_scores2 = train_model(model, train_loader, val_loader, criterion, optim.Adam, 0.001, 15)


Epoch 1/20, Train Accuracy: 0.5844, Val Accuracy: 0.6805
Epoch 2/20, Train Accuracy: 0.7206, Val Accuracy: 0.5734
Epoch 3/20, Train Accuracy: 0.7648, Val Accuracy: 0.7642
Epoch 4/20, Train Accuracy: 0.7906, Val Accuracy: 0.8195
Epoch 5/20, Train Accuracy: 0.8152, Val Accuracy: 0.8263
Epoch 6/20, Train Accuracy: 0.8293, Val Accuracy: 0.7642
Epoch 7/20, Train Accuracy: 0.8389, Val Accuracy: 0.7363
Epoch 8/20, Train Accuracy: 0.8521, Val Accuracy: 0.8170
Epoch 9/20, Train Accuracy: 0.8492, Val Accuracy: 0.8527
Epoch 10/20, Train Accuracy: 0.8681, Val Accuracy: 0.8205
Epoch 11/20, Train Accuracy: 0.8746, Val Accuracy: 0.8170
Epoch 12/20, Train Accuracy: 0.8838, Val Accuracy: 0.8454
Epoch 13/20, Train Accuracy: 0.8911, Val Accuracy: 0.8454
Epoch 14/20, Train Accuracy: 0.8983, Val Accuracy: 0.8469
Epoch 15/20, Train Accuracy: 0.9071, Val Accuracy: 0.8708
Epoch 16/20, Train Accuracy: 0.9134, Val Accuracy: 0.8297
Epoch 17/20, Train Accuracy: 0.9252, Val Accuracy: 0.8190
Epoch 18/20, Train Accu

In [39]:
model = ResNet(img_channels=3, num_layers=50, num_classes=len(classes))
train_acc3, val_acc3, map_scores3 = train_model(model, train_loader, val_loader, criterion, optim.Adam, 0.001, 20)


Epoch 1/20, Train Accuracy: 0.5795, Val Accuracy: 0.6712
Epoch 2/20, Train Accuracy: 0.7238, Val Accuracy: 0.7290
Epoch 3/20, Train Accuracy: 0.7678, Val Accuracy: 0.7564
Epoch 4/20, Train Accuracy: 0.7979, Val Accuracy: 0.8033
Epoch 5/20, Train Accuracy: 0.8087, Val Accuracy: 0.7309
Epoch 6/20, Train Accuracy: 0.8181, Val Accuracy: 0.8068
Epoch 7/20, Train Accuracy: 0.8319, Val Accuracy: 0.7940
Epoch 8/20, Train Accuracy: 0.8383, Val Accuracy: 0.8102
Epoch 9/20, Train Accuracy: 0.8500, Val Accuracy: 0.8371
Epoch 10/20, Train Accuracy: 0.8592, Val Accuracy: 0.8434
Epoch 11/20, Train Accuracy: 0.8617, Val Accuracy: 0.8263
Epoch 12/20, Train Accuracy: 0.8613, Val Accuracy: 0.8386
Epoch 13/20, Train Accuracy: 0.8740, Val Accuracy: 0.7979
Epoch 14/20, Train Accuracy: 0.8851, Val Accuracy: 0.8635
Epoch 15/20, Train Accuracy: 0.8956, Val Accuracy: 0.8547
Epoch 16/20, Train Accuracy: 0.9009, Val Accuracy: 0.8444
Epoch 17/20, Train Accuracy: 0.9068, Val Accuracy: 0.8302
Epoch 18/20, Train Accu

In [41]:
model = ResNet(img_channels=3, num_layers=101, num_classes=len(classes))
train_acc2, val_acc2 = train_model(model, train_loader, val_loader, criterion, optim.Adam, 0.001, 10)


Epoch 1/10, Train Accuracy: 0.5184, Val Accuracy: 0.6292
Epoch 2/10, Train Accuracy: 0.6608, Val Accuracy: 0.7167
Epoch 3/10, Train Accuracy: 0.7430, Val Accuracy: 0.7515
Epoch 4/10, Train Accuracy: 0.7653, Val Accuracy: 0.7304
Epoch 5/10, Train Accuracy: 0.7791, Val Accuracy: 0.7432
Epoch 6/10, Train Accuracy: 0.8047, Val Accuracy: 0.7877
Epoch 7/10, Train Accuracy: 0.8056, Val Accuracy: 0.7970
Epoch 8/10, Train Accuracy: 0.8246, Val Accuracy: 0.8322
Epoch 9/10, Train Accuracy: 0.8281, Val Accuracy: 0.8033
Epoch 10/10, Train Accuracy: 0.8385, Val Accuracy: 0.7099


In [42]:
model = ResNet(img_channels=3, num_layers=152, num_classes=len(classes))
train_acc2, val_acc2 = train_model(model, train_loader, val_loader, criterion, optim.Adam, 0.001, 10)


Epoch 1/10, Train Accuracy: 0.4795, Val Accuracy: 0.5744
Epoch 2/10, Train Accuracy: 0.6082, Val Accuracy: 0.6561
Epoch 3/10, Train Accuracy: 0.6937, Val Accuracy: 0.7343
Epoch 4/10, Train Accuracy: 0.7464, Val Accuracy: 0.7774
Epoch 5/10, Train Accuracy: 0.7799, Val Accuracy: 0.8180
Epoch 6/10, Train Accuracy: 0.7837, Val Accuracy: 0.7842
Epoch 7/10, Train Accuracy: 0.8001, Val Accuracy: 0.7676
Epoch 8/10, Train Accuracy: 0.7709, Val Accuracy: 0.7236
Epoch 9/10, Train Accuracy: 0.8006, Val Accuracy: 0.7999
Epoch 10/10, Train Accuracy: 0.8244, Val Accuracy: 0.7989
